<a href="https://colab.research.google.com/github/jgiganto/MyPyCaptchaSolver/blob/main/captchas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import files
import shutil
#shutil.rmtree('/content/dataset/captchas')


os.makedirs('/content/dataset/captchas', exist_ok=True)
# Subir el archivo ZIP desde tu computadora (selecciona dataset.zip)
uploaded = files.upload()

# Verifica que el archivo se haya subido; por defecto se guarda en /content


Saving dataset50kazu2.zip to dataset50kazu2.zip


In [ ]:
import shutil

# Mover el archivo ZIP a la carpeta /content/dataset/captchas
# Asegurate de que el archivo se llame "dataset.zip"
shutil.move('dataset50kazu2.zip', '/content/dataset/captchas/dataset50kazu2.zip')
#shutil.move('dataset2.zip', '/content/dataset/captchas/dataset2.zip')

# Descomprimir el archivo ZIP en la misma carpeta
!unzip /content/dataset/captchas/dataset50kazu2.zip -d /content/dataset/captchas
#!unzip /content/dataset/captchas/dataset2.zip -d /content/dataset/captchas


Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/dataset/captchas/91627-26106B.png  
  inflating: /content/dataset/captchas/9163-14846B.png  
  inflating: /content/dataset/captchas/91633-46545B.png  
  inflating: /content/dataset/captchas/91635-1566.png  
  inflating: /content/dataset/captchas/91635-29090B.png  
  inflating: /content/dataset/captchas/91636-12435B.png  
  inflating: /content/dataset/captchas/91637-24730B.png  
  inflating: /content/dataset/captchas/91639-37487B.png  
  inflating: /content/dataset/captchas/91640-18032B.png  
  inflating: /content/dataset/captchas/91641-12437B.png  
  inflating: /content/dataset/captchas/91641-22868B.png  
  inflating: /content/dataset/captchas/91642-39456B.png  
  inflating: /content/dataset/captchas/91643-11497B.png  
  inflating: /content/dataset/captchas/91643-15255B.png  
  inflating: /content/dataset/captchas/91645-11941B.png  
  inflating: /content/dataset/captchas/91646-13923B.png  
  inflating: /

In [ ]:
import os

# Define la ruta de la carpeta
folder_path = "/content/dataset/captchas"

# Listar todos los archivos (excluyendo subcarpetas)
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Imprimir el número total de archivos
print("Total de archivos en la carpeta:", len(files))


Total de archivos en la carpeta: 51999


In [ ]:
# Si es necesario, instala las librerías (en Colab ya vienen la mayoría, pero para estar seguros)
!pip install opencv-python scikit-learn

# (Opcional) Si tus datos están en Google Drive, descomenta las siguientes líneas:
# from google.colab import drive
# drive.mount('/content/drive')
# Ajusta DATASET_PATH si lo usas desde Drive, por ejemplo:
# DATASET_PATH = "/content/drive/MyDrive/dataset/captchas"

import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
# --- Se importan capas adicionales para la arquitectura CRNN ---
from tensorflow.keras.layers import Reshape, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Parámetros
IMG_WIDTH, IMG_HEIGHT = 225, 150
SEQ_LENGTH = 5  # Número máximo de dígitos por captcha
NUM_CLASSES = 11  # 10 dígitos (0 al 9) + 1 clase extra para 'blank'
CHARACTERS = "0123456789"
BLANK_TOKEN = 10  # Índice para 'blank'
DATASET_PATH = "/content/dataset/captchas"  # Asegúrate de que tus imágenes estén en esta ruta

# Función para cargar datos (se espera que las imágenes estén en blanco y negro,
# donde los números son blancos y el fondo es negro)
def load_data(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".png"):
            img_path = os.path.join(data_dir, filename)
            # Leer la imagen en escala de grises
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            # Redimensionar la imagen
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            # Aplicar umbral para obtener imagen binaria (0 o 1):
            # Se asigna 1 a los números (blancos) y 0 al fondo (negro)
            _, img = cv2.threshold(img, 127, 1, cv2.THRESH_BINARY)
            img = img.astype(np.float32)
            # Extraer el captcha: se toma la parte izquierda del guion del nombre del archivo
            label_text = filename.split("-")[0]
            # Convertir cada dígito en su índice correspondiente
            label = [CHARACTERS.index(char) for char in label_text]
            # Truncar si hay más de SEQ_LENGTH dígitos, o aplicar padding si es menor
            if len(label) > SEQ_LENGTH:
                label = label[:SEQ_LENGTH]
            elif len(label) < SEQ_LENGTH:
                label += [BLANK_TOKEN] * (SEQ_LENGTH - len(label))
            images.append(img)
            labels.append(label)
    images = np.array(images).reshape(-1, IMG_HEIGHT, IMG_WIDTH, 1)
    labels = np.array(labels)  # Se espera (num_samples, SEQ_LENGTH)
    if labels.shape[0] != images.shape[0]:
        labels = labels.T
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)
    return images, labels

# Cargar datos
X, y = load_data(DATASET_PATH)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Preparar etiquetas para multi-output en formato de diccionario
y_train_dict = {f'char_{i+1}': y_train[:, i] for i in range(SEQ_LENGTH)}
y_val_dict   = {f'char_{i+1}': y_val[:, i] for i in range(SEQ_LENGTH)}

# Data augmentation (se aplicará solo sobre X)
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1
)

# Generador personalizado que combina data augmentation y etiquetas en diccionario
def data_generator(X, y_dict, batch_size):
    n = X.shape[0]
    indices = np.arange(n)
    while True:
        np.random.shuffle(indices)
        for i in range(0, n, batch_size):
            batch_idx = indices[i:i+batch_size]
            X_batch = X[batch_idx]
            # Aplicar data augmentation
            X_batch = next(datagen.flow(X_batch, batch_size=X_batch.shape[0], shuffle=False))
            y_batch = {k: v[batch_idx] for k, v in y_dict.items()}
            yield X_batch, y_batch

# --- Cambio: Aumentamos el batch_size de 32 a 64 para mayor estabilidad ---
batch_size = 256
steps_per_epoch = int(np.ceil(X_train.shape[0] / batch_size))
validation_steps = int(np.ceil(X_val.shape[0] / batch_size))

# --- Cambio: Se reemplaza la arquitectura CNN original por una arquitectura CRNN (CNN + LSTM) ---
def create_crnn_model():
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1))

    # Bloque 1
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Bloque 2
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Bloque 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Bloque 4 - añadido para una arquitectura más profunda
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Transformar la salida para la capa recurrente (se toma el ancho como dimensión temporal)
    shape = tf.keras.backend.int_shape(x)  # (None, height, width, channels)
    # Se reconfigura a (batch, time_steps=width, features=height*channels)
    x = Reshape((shape[2], shape[1] * shape[3]))(x)

    # Capa Bidirectional LSTM para capturar la secuencia de características
    x = Bidirectional(LSTM(128, return_sequences=False))(x)

    # Capas densas finales
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Salidas para cada dígito del captcha
    outputs = []
    for i in range(SEQ_LENGTH):
        out = Dense(NUM_CLASSES, activation='softmax', name=f'char_{i+1}')(x)
        outputs.append(out)

    model = Model(inputs=inputs, outputs=outputs)
    loss_dict = {f'char_{i+1}': 'sparse_categorical_crossentropy' for i in range(SEQ_LENGTH)}
    metrics_dict = {f'char_{i+1}': 'accuracy' for i in range(SEQ_LENGTH)}
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss=loss_dict,
                  metrics=metrics_dict)
    return model

model = create_crnn_model()
model.summary()

# Callback personalizado que detiene el entrenamiento si el accuracy supera 0.93 en cada salida
class EarlyStoppingByAcc(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Comprobar el accuracy de cada salida
        accuracies = [logs.get(f'char_{i+1}_accuracy') for i in range(SEQ_LENGTH)]
        if all(acc is not None and acc >= 0.98 for acc in accuracies):
            print(f"\nTodos los accuracies han superado 0.98 en la época {epoch+1}. Se detiene el entrenamiento.")
            self.model.stop_training = True

# Callbacks: early stopping, reducción del learning rate y callback personalizado
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

acc_early_stopping = EarlyStoppingByAcc()

train_gen = data_generator(X_train, y_train_dict, batch_size)
val_gen = data_generator(X_val, y_val_dict, batch_size)

history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=2000,
    validation_data=val_gen,
    validation_steps=validation_steps,
    callbacks=[early_stopping, reduce_lr, acc_early_stopping]
)

# Guardar el modelo en Colab (o en Drive si montaste tu drive)
MODEL_FOLDER = "saved_models"
os.makedirs(MODEL_FOLDER, exist_ok=True)
MODEL_PATH = os.path.join(MODEL_FOLDER, "model_tf.keras")
model.save(MODEL_PATH)
print(f"Modelo guardado en {MODEL_PATH}")

# Comprimir y descargar el modelo
!zip -r model_tf.zip {MODEL_FOLDER}
from google.colab import files
files.download("model_tf.zip")


Images shape: (51997, 150, 225, 1)
Labels shape: (51997, 5)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 150, 225, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 150, 225, 32)   │            320 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 150, 225, 32)   │            128 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 75, 112, 32)    │              0 │ batch_normalization[0… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 75, 112, 64)    │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 75, 112, 64)    │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 37, 56, 64)     │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 37, 56, 128)    │         73,856 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 37, 56, 128)    │            512 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 18, 28, 128)    │              0 │ batch_normalization_2… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 18, 28, 256)    │        295,168 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 18, 28, 256)    │          1,024 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 9, 14, 256)     │              0 │ batch_normalization_3… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 14, 2304)       │              0 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 256)            │      2,491,392 │ reshape[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├──────────────────────

 Total params: 2,961,079 (11.30 MB)

 Trainable params: 2,960,119 (11.29 MB)

 Non-trainable params: 960 (3.75 KB)

Epoch 1/2000
183/183 ━━━━━━━━━━━━━━━━━━━━ 150s 745ms/step - char_1_accuracy: 0.1070 - char_1_loss: 2.3476 - char_2_accuracy: 0.1002 - char_2_loss: 2.3928 - char_3_accuracy: 0.1025 - char_3_loss: 2.3889 - char_4_accuracy: 0.1073 - char_4_loss: 2.3870 - char_5_accuracy: 0.1247 - char_5_loss: 2.3942 - loss: 11.9105 - val_char_1_accuracy: 0.1152 - val_char_1_loss: 2.3116 - val_char_2_accuracy: 0.0971 - val_char_2_loss: 2.3426 - val_char_3_accuracy: 0.1067 - val_char_3_loss: 2.3486 - val_char_4_accuracy: 0.0921 - val_char_4_loss: 2.3671 - val_char_5_accuracy: 0.1012 - val_char_5_loss: 2.4553 - val_loss: 11.8280 - learning_rate: 1.0000e-04
Epoch 2/2000
183/183 ━━━━━━━━━━━━━━━━━━━━ 130s 709ms/step - char_1_accuracy: 0.1381 - char_1_loss: 2.2446 - char_2_accuracy: 0.1155 - char_2_loss: 2.3468 - char_3_accuracy: 0.1272 - char_3_loss: 2.3199 - char_4_accuracy: 0.1599 - char_4_loss: 2.2887 - char_5_accuracy: 0.2322 - char_5_loss: 2.2051 - loss: 11.4051 - val_char_1_accuracy: 0.1419 - val_char_1_l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>